## Installs

In [1]:
!pip install impyla
!pip install ibis-framework 
!pip install ibis-framework[impala]

## Load Libs

In [2]:
#from impala.dbapi import connect

In [3]:
import numpy as np
import math
import pandas as pd
import datetime as dt
import ibis

In [4]:
impala_server = 'ec2-54-252-190-231.ap-southeast-2.compute.amazonaws.com'

## Generate Dataframe

In [5]:
windVal = []
windError = []
windOrientation = []
prevVal = 20
prevOrientation = np.random.uniform(0, 360)
for i in range(0, 86400):
    windVal.append(abs(np.random.normal(prevVal, 2, 1)[0]))
    windError.append(abs(np.random.normal(round(prevVal/10), 1)))
    if(i % 100 == 0):
        windOrientation.append(np.random.uniform(prevOrientation-50,
                                                 prevOrientation+50))
    else:
        windOrientation.append(np.random.uniform(prevOrientation-5,
                                                 prevOrientation+5))
    if(round(windVal[-1]) > 45):
        prevVal = int(math.floor(windVal[-1]))
    elif(round(windVal[-1]) < 10):
        prevVal = int(math.ceil(windVal[-1]))
    else:
        prevVal = int(round(windVal[-1]))
    prevOrientation = windOrientation[-1]

In [6]:
df = pd.DataFrame.from_dict({
                             'speed': windVal,
                             'speedError': windError,
                             'direction': windOrientation
                            })

In [7]:
now = dt.datetime.now()
sec = now.second
minute = now.minute
hour = now.hour

totalTime = (hour * 3600) + (minute * 60) + (sec)

In [8]:
df['rowid'] = df.index

## Interfacing with Impala

In [9]:
hdfs = ibis.hdfs_connect(host=impala_server, port=9870, protocol='webhdfs')
con = ibis.impala.connect(host=impala_server, database='default', hdfs_client=hdfs)

In [10]:
db = con.database('default')
db.create_table('wind', df)

OperationalError: Operation is in ERROR_STATE

# Testing Script

In [13]:
%ls ../db

api.py                  impala_api.py  load_data.py
build_and_load_data.py  __init__.py    wind-data.db


In [27]:
%env IMPALA_HOST=ec2-13-238-118-193.ap-southeast-2.compute.amazonaws.com
%env IMPALA_PORT=21050
%env WEBHDFS_HOST=ec2-13-238-118-193.ap-southeast-2.compute.amazonaws.com
%env WEBHDFS_PORT=9870

env: IMPALA_HOST=ec2-13-238-118-193.ap-southeast-2.compute.amazonaws.com
env: IMPALA_PORT=21050
env: WEBHDFS_HOST=ec2-13-238-118-193.ap-southeast-2.compute.amazonaws.com
env: WEBHDFS_PORT=9870


In [17]:
import math
import numpy as np
import pandas as pd
import ibis
import hdfs
import os

In [28]:
impala_host = os.environ['IMPALA_HOST']
#impala_host = 'ec2-13-238-118-193.ap-southeast-2.compute.amazonaws.com'
impala_port = int(os.environ['IMPALA_PORT'])
webhdfs_host = os.environ['WEBHDFS_HOST']
#webhdfs_host = impala_host
webhdfs_port = int(os.environ['WEBHDFS_PORT'])

In [29]:
os.environ['IMPALA_HOST']

'ec2-13-238-118-193.ap-southeast-2.compute.amazonaws.com'

In [30]:
hdfs = ibis.hdfs_connect(host=webhdfs_host, port=webhdfs_port)
client = ibis.impala.connect(
    host=impala_host, port=impala_port, hdfs_client=hdfs
)

In [31]:
%run ../db/build_and_load_data.py

building frame
converting parquet


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
pyarrow or fastparquet is required for parquet support